In [10]:
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, Accuracy
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD, Adam,schedules
from sklearn.metrics import confusion_matrix
import numpy as np




In [11]:
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
import random
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt
TRAIN_SPLIT = 0.85

DIR = './'
imgH = 32
imgW = 32

# file = open('data_index.txt', 'w')
count = 0

def load_image_data():
    imgDir = DIR + 'Normal/'
    imgSet1 = prepare_image_array(imgDir, imgW, imgH)
    m = imgSet1.shape[0]

    imgDir = DIR + 'OverExposed/'
    imgSet2 = prepare_image_array(imgDir, imgW, imgH)
    n = imgSet2.shape[0]

    imgDir = DIR + 'UnderExposed/'
    imgSet3 = prepare_image_array(imgDir, imgW, imgH)
    o = imgSet3.shape[0]

    # Put all image data into one array.
    imgSet = np.concatenate((imgSet1, imgSet2, imgSet3), axis=0)
    print(imgSet.shape)

    labelSet1 = np.zeros(m, dtype=np.uint8)
    labelSet2 = np.ones(n, dtype=np.uint8)
    label_o = np.ones(o, dtype=np.uint8)
    labelSet3 = np.add(label_o, label_o)
    labelSet = np.concatenate((labelSet1, labelSet2, labelSet3), axis=0)


    p = imgSet.shape[0]  # p = n + m + o

    # random.shuffle(indices)
    # for i in range(len(indices)):
    #     file.write(str(indices[i])+",")
    
    # file.close()
    f = open('data_index.txt', 'r')
    indices = []
    for x in f.read().split(','):
        if len(x) > 0:
            num = ''
            for j in x:
                if(j>='0' and j<='9'):
                    num +=j;
            if len(num)>0 and  int(num)<p:
                indices.append(int(num)) 
   
    # print(indices)
    imgSet = imgSet[indices]
    labelSet = labelSet[indices]
    print(labelSet[:5])
    print("Label set : ")
    labelSet = labelSet.reshape(p, 1)

#   one hot encoding

    labelSet = to_categorical(labelSet)

    # print(labelSet[:5])
    
    # print(imgSet[0])
    imgSet = imgSet.astype(np.float32)
    imgSet = imgSet/255.0

    r = int(p * TRAIN_SPLIT)
    trainX = imgSet[:r]
    trainY = labelSet[:r]
    testX = imgSet[r:]
    testY = labelSet[r:]

    return trainX, trainY, testX, testY



def prepare_image_array(imgDir, imgW, imgH):
    imgList = os.listdir(imgDir)
    # print(imgList)
    n = len(imgList)

    imgSet = []
    for i in range(n):
        imgPath = imgDir + imgList[i]
        if (os.path.exists(imgPath)):
            # print(imgPath)
            img = cv2.imread(imgPath)
            resizedImg = cv2.resize(img, (imgW, imgH))
            rgbImg = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2RGB)
            imgSet.append(rgbImg)
        else:
            print("It is not a valid image path.")

    # print("total image "+str(len(imgSet)))
    imgSet = np.array(imgSet, dtype=np.uint8)
    # print("total shape "+str(imgSet.shape))

    return imgSet



In [12]:
train_x,train_Y, testInSet, testOutSet =load_image_data()


(8910, 32, 32, 3)
[0 2 1 2 0]
Label set : 


In [13]:
from sklearn.metrics import classification_report

def run_model(modelpath,testInSet, testOutSet):

    lr_schedule = schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=10000,
        decay_rate=0.9)

    sgd = SGD(learning_rate=lr_schedule)


    model = load_model(modelpath, compile = True)
    model.compile(loss = 'mse', optimizer = sgd, metrics = ['accuracy', Recall(), Precision()])
    


#     y_test = np.argwhere(testOutSet)[:,1]
    # print(y_test)
#     indices = np.argwhere(y_test == 0).flatten()
#     print(indices.shape)
#     testInNormal = testInSet[indices]
#     testOutNormal = testOutSet[indices]



#     indices = np.argwhere(y_test == 1).flatten()
#     testInOver = testInSet[indices]
#     testOutOver = testOutSet[indices]
#     indices = np.argwhere(y_test == 2).flatten()
#     testInUnder = testInSet[indices]
#     testOutUnder = testOutSet[indices]



    '''	Test Model. '''
    lossT, accT, precisionT, recallT = model.evaluate(testInSet, testOutSet)
#     lossN, accN, precisionN, recallN = model.evaluate(testInNormal, testOutNormal)
#     lossO, accO, precisionO, recallO = model.evaluate(testInOver, testOutOver)
#     lossU, accU, precisionU, recallU = model.evaluate(testInUnder, testOutUnder)


    print("\n\nlossT = %s, accT = %s,  precisionT = %s, recallT = %s "%(lossT, accT, precisionT, recallT))
#     print("lossN = %s, accN = %s,  precisionN = %s, recallN = %s "%(lossN, accN, precisionN, recallN))
#     print("lossO = %s, accO = %s,  precisionO = %s, recallO = %s "%(lossO, accO, precisionO, recallO))
#     print("lossU = %s, accU = %s,  precisionU = %s, recallU = %s "%(lossU, accU, precisionU, recallU))

    predicted_output = []
    prediction = model.predict(testInSet)
    for i in prediction:
        predicted_output.append(np.argmax(i)) 

    actual_output = []
    for i in testOutSet:
        actual_output.append(np.argmax(i)) 
    con_mat = confusion_matrix(actual_output, predicted_output)
#     for i in con_mat:
#         print(i)
        
    cm =  classification_report(actual_output,predicted_output )
#     print(cm)
    cm = cm.split(" ")
    count = 0
    cnt = 0
    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "
    acc = "Accuracy & \multicolumn{3}{r}{"
    mac = "Macro Avg "
    weight = "Weighted Avg "
    for i in cm:
        if len(i)==4:
            count = 0
            for j in i:
                if j=='.':
                    count =1
            if count ==1:
                if(cnt<3):
                    first_line +=" & %s"%i[2:]
                    first_line +='\%'
                if cnt>=3 and cnt<6:
                    second_line += " & %s"%i[2:]
                    second_line +='\%'
                if cnt>=6 and  cnt<9:
                    third_line += " & %s"%i[2:]
                    third_line +='\%'
                if cnt>=9 and cnt<10:
                    acc += " %s"%i[2:]
                    acc +='\%}'
                if cnt>=10 and cnt<13:
                    mac +=" & %s"%i[2:]
                    mac +='\%'
                if cnt>=13 and cnt<16:
                    weight += " & %s"%i[2:]
                    weight +='\%'
                cnt+=1

    print(first_line+'\\\\')
    print(second_line+'\\\\')
    print(third_line+'\\\\\\hline')
    print(acc+'\\\\')
    print(mac+'\\\\')
    print(weight+'\\\\\hline\hline\\thinlines')



    print("\n\n\n\n")


    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "

    count = 0
    for i in con_mat:
        if count==0:
            first_line+= ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==1:
            second_line+=  ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==2:
            third_line+=   ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        count+=1


    print(first_line+" \\\\\hline")
    print(second_line+" \\\\\hline")
    print(third_line+" \\\\\hline")

    
    return cm,con_mat

    

In [14]:
print("Non trainable")
DIR = './non-trainable/'
modelpath = DIR + 'VGG_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Non trainable
42/42 [==============================] - 4s 75ms/step - loss: 0.2214 - accuracy: 0.4383 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00


lossT = 0.22138726711273193, accT = 0.4382946789264679,  precisionT = 0.0, recallT = 0.0 
Normal  & 37\% & 53\% & 43\%\\
Over Expsoed  & 60\% & 16\% & 25\%\\
Under Expsoed  & 48\% & 62\% & 54\%\\\hline
Accuracy & \multicolumn{3}{r}{ 44\%}\\
Macro Avg  & 48\% & 44\% & 41\%\\
Weighted Avg  & 49\% & 44\% & 41\%\\\hline\hline\thinlines





Normal  & 222 & 28 & 167 \\\hline
Over Expsoed  & 224 & 70 & 154 \\\hline
Under Expsoed  & 160 & 18 & 294 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.37',
  '',
  '',
  '',
  '',
  '',
  '0.53',
  '',
  '',
  '',
  '',
  '',
  '0.43',
  '',
  '',
  '',
  '',
  '',
  '',
  '417\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.60',
  '',
  '',
  '',
  '',
  '',
  '0.16',
  '',
  '',
  '',
  '',
  '',
  '0.25',
  '',
  '',
  '',
  '',
  '',
  '',
  '448\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.48',
  '',
  '',
  '',
  '',
  '',
  '0.62',
  '',
  '',
  '',
  '',
  '',
  '0.54',
  '',
  '',
  '',
  '',
  '',
  '',
  '472\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [15]:
print("trainable")
DIR = './trainable/'
modelpath = DIR + 'trainable-tf-Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

trainable
42/42 [==============================] - 3s 72ms/step - loss: 0.1318 - accuracy: 0.7315 - recall_6: 0.6709 - precision_6: 0.7667


lossT = 0.13180872797966003, accT = 0.7314884066581726,  precisionT = 0.6709049940109253, recallT = 0.7666666507720947 
Normal  & 61\% & 54\% & 57\%\\
Over Expsoed  & 78\% & 76\% & 77\%\\
Under Expsoed  & 78\% & 87\% & 82\%\\\hline
Accuracy & \multicolumn{3}{r}{ 73\%}\\
Macro Avg  & 72\% & 72\% & 72\%\\
Weighted Avg  & 73\% & 73\% & 73\%\\\hline\hline\thinlines





Normal  & 226 & 92 & 99 \\\hline
Over Expsoed  & 89 & 340 & 19 \\\hline
Under Expsoed  & 56 & 4 & 412 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.61',
  '',
  '',
  '',
  '',
  '',
  '0.54',
  '',
  '',
  '',
  '',
  '',
  '0.57',
  '',
  '',
  '',
  '',
  '',
  '',
  '417\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.78',
  '',
  '',
  '',
  '',
  '',
  '0.76',
  '',
  '',
  '',
  '',
  '',
  '0.77',
  '',
  '',
  '',
  '',
  '',
  '',
  '448\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.78',
  '',
  '',
  '',
  '',
  '',
  '0.87',
  '',
  '',
  '',
  '',
  '',
  '0.82',
  '',
  '',
  '',
  '',
  '',
  '',
  '472\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [16]:
print("fully connected ")
DIR = './Fully-Connnected-model/'
modelpath = DIR + 'Fully_connected_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

fully connected 
42/42 [==============================] - 1s 4ms/step - loss: 0.1508 - accuracy: 0.6806 - recall_7: 0.6799 - precision_7: 0.6814


lossT = 0.15081512928009033, accT = 0.6806282997131348,  precisionT = 0.6798803210258484, recallT = 0.6814092993736267 
Normal  & 52\% & 65\% & 57\%\\
Over Expsoed  & 89\% & 46\% & 61\%\\
Under Expsoed  & 75\% & 92\% & 82\%\\\hline
Accuracy & \multicolumn{3}{r}{ 68\%}\\
Macro Avg  & 72\% & 68\% & 67\%\\
Weighted Avg  & 72\% & 68\% & 67\%\\\hline\hline\thinlines





Normal  & 269 & 24 & 124 \\\hline
Over Expsoed  & 216 & 208 & 24 \\\hline
Under Expsoed  & 36 & 3 & 433 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.52',
  '',
  '',
  '',
  '',
  '',
  '0.65',
  '',
  '',
  '',
  '',
  '',
  '0.57',
  '',
  '',
  '',
  '',
  '',
  '',
  '417\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.89',
  '',
  '',
  '',
  '',
  '',
  '0.46',
  '',
  '',
  '',
  '',
  '',
  '0.61',
  '',
  '',
  '',
  '',
  '',
  '',
  '448\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.75',
  '',
  '',
  '',
  '',
  '',
  '0.92',
  '',
  '',
  '',
  '',
  '',
  '0.82',
  '',
  '',
  '',
  '',
  '',
  '',
  '472\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [17]:
print("smalll")
DIR = './Small-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Small_CNN_Classifier.hdf5'

cm,con_mat = run_model(modelpath,testInSet, testOutSet)




smalll
42/42 [==============================] - 0s 4ms/step - loss: 0.1220 - accuracy: 0.7300 - recall_8: 0.7173 - precision_8: 0.7377


lossT = 0.12199132144451141, accT = 0.7299925088882446,  precisionT = 0.717277467250824, recallT = 0.7376922965049744 
Normal  & 59\% & 58\% & 58\%\\
Over Expsoed  & 80\% & 77\% & 79\%\\
Under Expsoed  & 79\% & 82\% & 81\%\\\hline
Accuracy & \multicolumn{3}{r}{ 73\%}\\
Macro Avg  & 72\% & 72\% & 72\%\\
Weighted Avg  & 73\% & 73\% & 73\%\\\hline\hline\thinlines





Normal  & 241 & 83 & 93 \\\hline
Over Expsoed  & 90 & 346 & 12 \\\hline
Under Expsoed  & 80 & 3 & 389 \\\hline


In [18]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Large-Size-Convolution-Maxpooling-model


ValueError: in user code:

    File "C:\Users\pc5\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\pc5\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pc5\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\pc5\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1471, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\pc5\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\pc5\AppData\Loca

###### 

In [ ]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)a  = {}